In [1]:
import os
import random

import torch
from torch.utils.data import DataLoader
from miditok import REMI, TokenizerConfig, MIDITokenizer, TokSequence
from miditok.pytorch_data import DatasetMIDI, DatasetJSON, DataCollator, split_midis_for_training
from miditok.data_augmentation import augment_midi_dataset
from miditok.utils import get_midi_programs
from pathlib import Path
from symusic import Score
import wandb
from tqdm import tqdm

from transformers.models.opt.modeling_opt import OPTForCausalLM

/home/lklimkiewicz/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Tokenizer

In [2]:
midi_paths = list(Path("/home/lklimkiewicz/priv/midi/data").glob("**/*.mid"))

In [3]:
tokenizer_config = TokenizerConfig(num_velocities=16, use_chords=True, use_programs=True)
tokenizer = REMI(tokenizer_config)

In [4]:
random.shuffle(midi_paths)
tokenizer.learn_bpe(vocab_size=30000, files_paths=midi_paths[:1000])

In [5]:
# saving
tokenizer.save_pretrained('../logs/tokenizer2')

In [3]:
# loading
tokenizer = MIDITokenizer.from_pretrained('../logs/tokenizer2')

config.json not found in /home/lklimkiewicz/priv/midi/logs/tokenizer2


## Model

In [4]:
from transformers import AutoConfig, AutoModel, AutoModelForCausalLM

In [6]:
config = AutoConfig.from_pretrained(
    'facebook/opt-125m',
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None'],
    pad_token_id=tokenizer['PAD_None'],
    vocab_size=len(tokenizer),
    prefix=None,
    max_length=1024,
    do_sample=True,
)

In [7]:
model = AutoModelForCausalLM.from_config(config)

## Split dataset

In [6]:
midi_paths = list(Path("/home/lklimkiewicz/priv/midi/data").glob("**/*.mid"))

In [8]:
def filter_dataset(paths, val_fun):
    correct = []
    for path in tqdm(paths):
        try:
            midi = Score(path)
            if val_fun(midi):
                correct.append(path)
        except:
            os.remove(path)
    return correct

In [9]:
print('Initial count:', len(midi_paths))

def midi_valid(midi) -> bool:
    if midi.note_num() < 50 or len(midi.time_signatures) == 0 or len(midi.tempos) == 0:
        return False
    
    for time_sig in midi.time_signatures:
        if time_sig.denominator == 0 or time_sig.numerator == 0:
            return False
    
    return True

midi_paths = filter_dataset(midi_paths, midi_valid)

print('Filtered count:', len(midi_paths))

Initial count: 1067913


100%|██████████| 1067913/1067913 [13:06<00:00, 1357.29it/s]


Filtered count: 766717


In [ ]:
split_midis_for_training(
    files_paths=midi_paths,
    tokenizer=tokenizer,
    save_dir=Path('./chunks_for_training'),
    max_seq_len=1024,
)

## Tokenize dataset

In [4]:
midi_paths = list(Path("/home/lklimkiewicz/priv/midi/src/chunks_for_training/khinsider").glob("**/*.mid"))
tokenizer.tokenize_midi_dataset(midi_paths, out_dir="tokenized_dataset/khinsider", save_programs=True)

Tokenizing MIDIs (tokenized_dataset/khinsider):  44%|████▍     | 42160/95341 [09:03<15:17, 57.94it/s] 

.

Tokenizing MIDIs (tokenized_dataset/khinsider):  44%|████▍     | 42198/95341 [09:04<16:28, 53.78it/s]

.

Tokenizing MIDIs (tokenized_dataset/khinsider): 100%|██████████| 95341/95341 [21:09<00:00, 75.09it/s] 


## Augment dataset

In [7]:
augment_midi_dataset(
    Path('/home/lklimkiewicz/priv/midi/src/tokenized_dataset'),
    pitch_offsets=[-12, 12],
    velocity_offsets=[-4, 5],
    duration_offsets=[-0.5, 1],
    out_path="./augmented_dataset",
)

Performing data augmentation: 0it [00:00, ?it/s]


## Load dataset

In [8]:
midi_paths = list(Path("/home/lklimkiewicz/priv/midi/src/chunks_for_training_2").glob("**/*.mid"))

dataset = DatasetMIDI(
    files_paths=midi_paths,
    tokenizer=tokenizer,
    max_seq_len=1024,
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
)

# json_paths = list(Path("/home/lklimkiewicz/priv/midi/src/tokenized_dataset").glob("**/*.json"))

# dataset = DatasetJSON(
#     files_paths=json_paths,
#     max_seq_len=1024,
#     bos_token_id=tokenizer["BOS_None"],
#     eos_token_id=tokenizer["EOS_None"],
# )

collator = DataCollator(
    tokenizer["PAD_None"],
    copy_inputs_as_labels=True,
    shift_labels=True,
)

## Train

In [9]:
from transformers import Trainer, TrainingArguments, TrainerCallback, TrainerState, TrainerControl

In [10]:
class MidiGenerationCallback(TrainerCallback):
    def on_step_end(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        if state.global_step % 500 == 0:
            
            input = torch.tensor([[tokenizer['BOS_None']]], device=model.device)
            midi = model.generate(input, max_new_tokens=1024)
            generated_ts = TokSequence(ids=midi.tolist()[0], ids_bpe_encoded=True)
            generated_score = tokenizer(generated_ts)
            generated_score.dump_midi(f'outputs/v1/step-{state.global_step}.mid')
            
            input = torch.tensor([[4]], device=model.device)
            midi = model.generate(input, max_new_tokens=1024)
            generated_ts = TokSequence(ids=midi.tolist()[0], ids_bpe_encoded=True)
            generated_score = tokenizer(generated_ts)
            generated_score.dump_midi(f'outputs/v2/step-{state.global_step}.mid')

In [11]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=4,
    report_to="wandb",
    bf16=True,
    dataloader_num_workers=24,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    lr_scheduler_type="cosine",
    warmup_steps=2000,
    save_steps=1000,
    save_total_limit=5,
    num_train_epochs=1,
    label_smoothing_factor=0.2,
    torch_compile=True,
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=collator,
    tokenizer=tokenizer,
    callbacks=[MidiGenerationCallback()]
)

In [13]:
os.environ["WANDB_PROJECT"] = "midi"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

In [15]:
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
/home/lklimkiewicz/miniconda3/envs/midi2/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
  2%|▏         | 21540/974907 [1:06:34<49:06:46,  5.39it/s]
/home/lklimkiewicz/miniconda3/envs/midi2/lib/python3.10/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get 

{'loss': 6.3445, 'learning_rate': 4.9952466897011097e-05, 'epoch': 0.02}


{'loss': 6.3532, 'learning_rate': 4.9951968025086607e-05, 'epoch': 0.02}


{'loss': 6.3451, 'learning_rate': 4.995146655143412e-05, 'epoch': 0.02}


{'loss': 6.4201, 'learning_rate': 4.995096247610592e-05, 'epoch': 0.02}


...

{'loss': 5.6733, 'learning_rate': 4.983959556590408e-05, 'epoch': 0.04}


{'loss': 5.7043, 'learning_rate': 4.9838681264173546e-05, 'epoch': 0.04}


{'loss': 5.6791, 'learning_rate': 4.9837764372527374e-05, 'epoch': 0.04}


{'loss': 5.743, 'learning_rate': 4.9836844891061155e-05, 'epoch': 0.04}


IndexError: Caught IndexError in DataLoader worker process 4.
Original Traceback (most recent call last):
  File "/home/lklimkiewicz/miniconda3/envs/midi2/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/lklimkiewicz/miniconda3/envs/midi2/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/lklimkiewicz/miniconda3/envs/midi2/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/lklimkiewicz/miniconda3/envs/midi2/lib/python3.10/site-packages/miditok/pytorch_data/datasets.py", line 187, in __getitem__
    tokseq = self._tokenize_midi(midi)
  File "/home/lklimkiewicz/miniconda3/envs/midi2/lib/python3.10/site-packages/miditok/pytorch_data/datasets.py", line 227, in _tokenize_midi
    tok

In [10]:
model.save_pretrained('./new_model', tokenizer=tokenizer)

NameError: name 'model' is not defined

In [16]:
tokenizer.save_pretrained('./tokenizer')

In [ ]:
wandb.finish()